In [1]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import Timeout
import xml.etree.ElementTree as ET
import json

ITEMS_COUNT = 100

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    
}

timeout_settings = (60, 60)

In [3]:
middle_category_nums = ['367', '596', '2074', '595', '980', '981', 
                        '1477', '657', '1004', '2080', '1003', '1002',
                        '2224']

In [4]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers, timeout=timeout_settings)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [5]:
def get_item_info(goods):
    # 상품 id
    goodsNo = goods['goodsNo']

    # 브랜드 명
    brandNm = goods['brandNm']

    # 정가
    norPrc = goods['norPrc']

    # 할인가
    salePrc = goods['salePrc']

    # 이미지    
    image_urls = []      
    try:    
        for i in range(len(goods['colorContInfo'])):
            colorContInfo = goods['colorContInfo'][i]['dispGoodsContUrl']
            image_urls.append(colorContInfo)
    except:
        pass

    # 색상 정보
    colors = []
    try:
        for color in goods['colorInfo']:
            colors.append(color['optnNm'])
    except:
        pass

    # 사이즈 정보
    sizes = []
    for i in range(len(goods['colorInfo'][0]['colorSizeInfo'])):
        size_info = goods['colorInfo'][0]['colorSizeInfo'][i]['erpSzCd']
        sizes.append(size_info)

    data = {
        'goodsNo': goodsNo,
        'brandNm': brandNm,
        'norPrce': norPrc,
        'salePrc': salePrc,
        'image_urls': image_urls,
        'colors': colors,
        'sizes': sizes
    }

    return data

In [6]:
def get_items(num):
    url = f'https://www.thehandsome.com/ko/DP/category/{num}?dispMediaCd=10&sortGbn=20&pageSize=21&pageNo=1&norOutletGbCd=J&dispCtgNo={num}&productListLayout=4&theditedYn=N&saleGoodsYn&soutYn&brandNo&strSalePrc&endSalePrc&colorNo&sizeNo&matCode'
    
    flag = 0
    page = 1

    goods_info = []

    while flag == 0:
        
        soup = get_response(url, headers)
        info = soup.string
        goods_in_page = json.loads(info)['payload']['goodsList']

        for goods in goods_in_page:
            
            goods_data = get_item_info(goods)
            goods_info.append(goods_data)
            
            if len(goods_info) == ITEMS_COUNT:
                break

        page += 1
        url = f'https://www.thehandsome.com/ko/DP/category/{num}?dispMediaCd=10&sortGbn=20&pageSize=21&pageNo={page}&norOutletGbCd=J&dispCtgNo={num}&productListLayout=4&theditedYn=N&saleGoodsYn&soutYn&brandNo&strSalePrc&endSalePrc&colorNo&sizeNo&matCode'

    return goods_info

In [7]:
def get_one_review(review):
    # 내용
    revCont = review['revCont']
    
    # 상품 id
    goodsNo = review['goodsNo']

    # 키
    try:
        height = review['revPrfleList'][0]['mbrPrfleValNm']
    except:
        height = None
    
    # 평소 사이즈
    try:
        nor_size = review['revPrfleList'][1]['mbrPrfleValNm']
    except:
        nor_size = None

    # 체형 정보
    form_info = {'height': height,
                 'nor_size': nor_size}

    data = {
        'goodsNo': goodsNo,
        'revCont': revCont,
        'form_info': form_info
    }

    return data

In [8]:
def get_review_data(goodsNo):
    url = f'https://www.thehandsome.com/api/goods/1/ko/goods/{goodsNo}/reviews?sortTypeCd=latest&revGbCd=&pageSize=10&pageNo=1'
    page = 1
    reviews = []
    while True:
        try:
            soup = get_response(url, headers)
            reviews = soup.string

            ten_reviews = json.loads(reviews)['payload']['revAllList']
            
            for review in ten_reviews:
                one_review = get_one_review(review)
                reviews.append(one_review)
            
            page += 1
            url = f'https://www.thehandsome.com/api/goods/1/ko/goods/{goodsNo}/reviews?sortTypeCd=latest&revGbCd=&pageSize=10&pageNo={page}'
            
        except:
            break

    return reviews

    

In [9]:
def main():
    all_info = []
    for num in middle_category_nums:
        middle_category_info = get_items(num)
        all_info.append(middle_category_info)
    
    goodsNos = []

    for info in all_info:
        goodsNos.append(info['goodsNo'])
        
    for goodsNo in goodsNos:
        reviews = get_review_data(goodsNo)

    df_all_info = pd.DataFrame(all_info)
    df_all_reviews = pd.DataFrame(reviews)

    df_all_info.to_csv('../all_info.csv', index=False)
    df_all_reviews.to_csv('../all_reviews.csv', index=False)

In [10]:
if __name__ == '__main__':
    main()

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))